# Data

- [Download corrupted Cifar10 .tar](https://zenodo.org/records/2535967) and extract it into a folder.

- Cifar10 is downloaded using torch.datasets

- [Download corrupted Tiny-ImageNet](https://zenodo.org/records/2536630) and extract it into a folder.

- [Download Tiny-ImageNet](https://www.kaggle.com/datasets/akash2sharma/tiny-imagenet) and extract it into a folder.

# Imports


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
import os
import timm

from my_transformers import CorruptDistillVisionTransformer
from utils import load_experimental_TinyImageNet

c:\Users\Hp\anaconda3\envs\deepLearning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Hp\anaconda3\envs\deepLearning\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


# Loading Data

In [8]:
corrupt_types = ["motion_blur", "shot_noise", "jpeg_compression", "fog"]

# corrupt_path = r"C:\Users\Hp\Desktop\Coding\Transformer-Thesis\Tiny-ImageNet-C\Tiny-ImageNet-C"
# normal_path = r"C:\Users\Hp\Desktop\Coding\Transformer-Thesis\Tiny-ImageNet-Normal"
# train_data, test_data = load_experimental_TinyImageNet(normal_path, corrupt_path, corrupt_types, num_train_imgs=20)


# Visualisation TinyImageNet

In [ ]:
def tensor_to_img(tensor):
    mean = torch.tensor([0.4802, 0.4481, 0.3975])
    std = torch.tensor([0.2302, 0.2265, 0.2262])
    
    # denormalize
    img = tensor.clone()
    img = img * std[:, None, None] + mean[:, None, None]
    img = torch.clamp(img, 0, 1)
    
    # convert to PIL image
    to_pil = T.ToPILImage()
    return to_pil(img)

# plot
imgs_to_display = [random.randint(0, len(train_data[0])-1) for i in range(9)]
fig, axes = plt.subplots(3, 3, figsize=(4, 4))
axes = axes.flatten()

for i in range(9):
    img, label = train_data[0][imgs_to_display[i]], train_data[2][imgs_to_display[i]].item()
    img = tensor_to_img(img)
    if label == len(corrupt_types):
        axes[i].set_title(f"normal", fontsize=8)
    else:
        axes[i].set_title(corrupt_types[label], fontsize=8)
    axes[i].imshow(img)
    axes[i].axis("off")

plt.tight_layout()
plt.show()

# Model config for TinyImageNet

In [2]:
# setting seed 
torch.cuda.manual_seed(22)
random.seed(22)
torch.manual_seed(22)

device = "cuda" if torch.cuda.is_available() else "cpu"

corrupt_types = ["motion_blur", "shot_noise", "jpeg_compression", "fog"]

# Hyper-parameters
PATCH_SIZE = 8
IMG_SIZE = 64
EMBED_DIM = 192
NUM_HEADS = 3
NUM_IMG_TYPES = len(corrupt_types)+1
NUM_ENCODERS = 12
NUM_CLASSES = 200
DROPOUT = 0
DROP_PATH = 0.1
ERASE_PROB = 0.25

In [3]:
# train_data = torch.load("train_data.pt", weights_only=True)
# test_data = torch.load("test_data.pt", weights_only=True)

BATCH_SIZE = 128
# train_loader = DataLoader(dataset=TensorDataset(*torch.load("train_data.pt", weights_only=True)), 
#                                   batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=TensorDataset(*torch.load("test_data.pt", weights_only=True)), 
                                 batch_size=BATCH_SIZE, shuffle=True)
itr = iter(test_loader)
train_batch = [next(itr)]
test_batch = [next(itr)]

# deit3_small_patch16_224.fb_in22k_ft_in1k -- 22M
# convnext_tiny.fb_in22k_ft_in1k -- 28M
teacher_model = timm.create_model('deit3_small_patch16_224.fb_in22k_ft_in1k', pretrained=True).cuda()
# teacher_model = timm.create_model('vit_small_patch16_224.augreg_in21k_ft_in1k', pretrained=True).cuda()
teacher_model.head = nn.Linear(in_features=384, out_features=NUM_CLASSES, bias=True).cuda()
num_params = sum(p.numel() for p in teacher_model.parameters())
print(num_params)
print(teacher_model.head)

21751496
Linear(in_features=384, out_features=200, bias=True)


In [ ]:
from train_test_module import compute_ece
import json

class TrainTest_TeacherFt:
    def __init__(self, model, train_batches, test_batches, num_img_types, device):
        self.model = model.to(device)
        self.train_batches = train_batches
        self.test_batches = test_batches
        self.num_img_types = num_img_types
        self.device = device

        self.all_test_metrics, self.all_train_metrics = [], []

    def test(self, print_metrics=False):
        top1_correct_preds, top5_correct_preds = 0, 0
        total_samples, total_ece, total_entropy = 0, 0, 0

        total_per_type = torch.zeros(self.num_img_types, device=self.device)
        top1_correct_per_type = torch.zeros(self.num_img_types, device=self.device)
        top5_correct_per_type = torch.zeros(self.num_img_types, device=self.device)

        self.model.eval()
        with torch.no_grad():
            for x_batch, y_batch, c_batch in self.test_batches:
                x_batch, y_batch, c_batch = x_batch.to(self.device), y_batch.to(self.device), c_batch.to(self.device)
                x_batch = F.interpolate(x_batch, size=(224, 224), mode='bilinear', align_corners=False)
                preds = self.model(x_batch)
                del x_batch 
                
                total_samples += y_batch.size(0)
                # top-1 acc
                top1_right = (torch.argmax(preds, dim=1) == y_batch)
                top1_correct_preds += top1_right.sum().item()
                # top-5 acc
                top5_right = torch.topk(preds, 5, dim=1).indices.eq(y_batch.unsqueeze(1)).any(dim=1)
                top5_correct_preds += top5_right.sum().item()
                # ECE loss
                total_ece += compute_ece(preds, y_batch, self.device)
                # Per-type acc
                for t in range(self.num_img_types):
                    mask = (c_batch == t)
                    total_per_type[t] += mask.sum()
                    top1_correct_per_type[t] += top1_right[mask].sum()
                    top5_correct_per_type[t] += top5_right[mask].sum()
                # entropy
                total_entropy += -torch.sum(torch.softmax(preds, dim=1) * torch.log_softmax(preds, dim=1), dim=1).sum().item()
                del y_batch, c_batch
                
        top1_acc_per_type = (top1_correct_per_type / total_per_type).tolist()
        test_metrics = {
            "top1_acc" : top1_correct_preds / total_samples, 
            "top5_acc" : top5_correct_preds / total_samples, 
            "top1_acc_per_type" : top1_acc_per_type,
            "top5_acc_per_type" :(top5_correct_per_type / total_per_type).tolist(), 
            "error_rate_per_type" : [1 - acc for acc in top1_acc_per_type],
            "ece" : total_ece / total_samples,
            "entropy" : total_entropy / total_samples
        }
        if print_metrics : print(f"Test-Accuracy:{test_metrics['top1_acc']:.2f}")
        return test_metrics
    
    def train(self, optimizer, save_path, num_epochs=1, print_metrics=False):
        total_samples, top1_correct_preds, loss_total = 0, 0, 0
        self.model.train()
        for epoch in range(1, num_epochs+1):
            print(f"------- Epoch {epoch} -------")
            for x_batch, y_batch, _ in self.test_batches:
                x_batch, y_batch = x_batch.to(self.device), y_batch.to(self.device)
                x_batch = F.interpolate(x_batch, size=(224, 224), mode='bilinear', align_corners=False)
                preds = self.model(x_batch)
                del x_batch
                total_samples += y_batch.size(0)
                # top-1 acc
                top1_right = (torch.argmax(preds, dim=1) == y_batch)
                top1_correct_preds += top1_right.sum().item()
                #loss
                loss = F.cross_entropy(preds, y_batch)
                # backprop
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_total += loss.item() * y_batch.size(0)
                del y_batch, 

            test_metrics = self.test()
            train_metrics = {
                "loss_total": loss_total/total_samples,
                "top1_acc" : top1_correct_preds / total_samples
                }
            if print_metrics : 
                print(f"train-loss: {train_metrics['loss_total']:.2f} -- train-acc: {train_metrics['top1_acc']:.2f} -- "
                      f"train-acc: {test_metrics['top1_acc']:.2f}"
                      )
            self.all_train_metrics.append(train_metrics)
            self.all_test_metrics.append(test_metrics)
        
        # save trained model
        if save_path:
            torch.save(self.model.state_dict(), f"{save_path}.pth")
            print(f"Model saved to {save_path}")
            
            with open(f"{save_path}_train_metrics.json", "w") as f1:
                json.dump(self.all_train_metrics, f1, indent=4)
            with open(f"{save_path}_test_metrics.json", "w") as f2:
                json.dump(self.all_test_metrics, f2, indent=4)

In [21]:
train_test_teacher = TrainTest_TeacherFt(teacher_model, train_batch, test_batch, NUM_IMG_TYPES, device)
optimizer = optim.AdamW(teacher_model.parameters(), lr=1e-4, weight_decay=0.05)
train_test_teacher.train(optimizer, "lulu")

------- Epoch 1 -------
Model saved to lulu


In [22]:
train_test_teacher.all_test_metrics

[{'top1_acc': 0.3671875,
  'top5_acc': 0.5859375,
  'top1_acc_per_type': [0.29032257199287415,
   0.5,
   0.3448275923728943,
   0.4000000059604645,
   0.30000001192092896],
  'top5_acc_per_type': [0.4516128897666931,
   0.75,
   0.5862069129943848,
   0.6499999761581421,
   0.5],
  'error_rate_per_type': [0.7096774280071259,
   0.5,
   0.6551724076271057,
   0.5999999940395355,
   0.699999988079071],
  'ece': 0.0017773996805772185,
  'entropy': 4.249937057495117}]

In [ ]:
from train_test_module import LossCalculator
class TrainTestdeit:
    def __init__(self, model, teacher_model, train_batches, test_batches, head_strategy, num_img_types,
                 device, save_path:str, print_metrics=False, n_bins=15
                 ):
        assert head_strategy > 0 and head_strategy <= 3
        self.model = model.to(device)
        self.teacher_model = teacher_model.to(device)
        self.train_batches = train_batches
        self.test_batches = test_batches
        self.head_strategy = head_strategy
        self.num_img_types = num_img_types
        self.save_path = save_path

        self.device = device
        self.print_metrics = print_metrics
        self.n_bins = n_bins

        self.all_test_metrics, self.all_train_metrics = [], []
    
    # testing function
    def test_cdeit_model(self):
        top1_correct_preds, top5_correct_preds = 0, 0
        top1_correct_corrs, total_samples, total_ece, total_entropy = 0, 0, 0, 0

        total_per_type = torch.zeros(self.num_img_types, device=self.device)
        top1_correct_per_type = torch.zeros(self.num_img_types, device=self.device)
        top5_correct_per_type = torch.zeros(self.num_img_types, device=self.device)

        self.model.eval()
        with torch.no_grad():
            for x_batch, y_batch, c_batch in self.test_batches:
                x_batch, y_batch, c_batch = x_batch.to(self.device), y_batch.to(self.device), c_batch.to(self.device)
                tokens = self.model(x_batch)
                
                if len(tokens) == 3 and self.head_strategy != 1:
                    preds = (tokens[0] + tokens[1] + self.model.output_head.corrupt_connection(tokens[2])) / 3
                else:
                    preds = (tokens[0] + tokens[1]) / 2
                    print("done preds!")
                
                total_samples += y_batch.size(0)
                # top-1 acc
                top1_right = (torch.argmax(preds, dim=1) == y_batch)
                top1_correct_preds += top1_right.sum().item()
                # top-5 acc
                top5_right = torch.topk(preds, 5, dim=1).indices.eq(y_batch.unsqueeze(1)).any(dim=1)
                top5_correct_preds += top5_right.sum().item()
                # ECE loss
                total_ece += compute_ece(preds, y_batch, self.device) * y_batch.size(0)
                # Per-type acc
                for t in range(self.num_img_types):
                    mask = (c_batch == t)
                    total_per_type[t] += mask.sum()
                    top1_correct_per_type[t] += top1_right[mask].sum()
                    top5_correct_per_type[t] += top5_right[mask].sum()
                # entropy
                total_entropy += -torch.sum(torch.softmax(preds, dim=1) * torch.log_softmax(preds, dim=1), dim=1).sum().item()
                # top-1 corruption classification acc
                if len(tokens) == 3 : top1_correct_corrs += (torch.argmax(tokens[2], dim=1) == c_batch).sum().item()              

        top1_acc_per_type = (top1_correct_per_type / total_per_type).tolist()
        test_metrics = {
            "top1_acc" : top1_correct_preds / total_samples, 
            "top5_acc" : top5_correct_preds / total_samples, 
            "top1_acc_per_type" : top1_acc_per_type,
            "top5_acc_per_type" :(top5_correct_per_type / total_per_type).tolist(), 
            "error_rate_per_type" : [1 - acc for acc in top1_acc_per_type],
            "ece" : total_ece / total_samples,
            "entropy" : total_entropy / total_samples
        }
        if top1_correct_corrs : test_metrics["top1_corrupt_acc"] = top1_correct_corrs / total_samples
        if self.print_metrics : print(f"Test-Accuracy:{test_metrics['top1_acc']:.2f}")
        return test_metrics

    # ------------ training function ------------
    def train_cdeit_model(self, optimizer, num_epochs=1):
        self.model.train()
        for epoch in range(1, num_epochs+1):
            loss_total, loss_corrupt, loss_distill, loss_cls = 0, 0, 0, 0
            total_samples, sim_cls_distill, sim_cls_corr = 0, 0, 0

            print(f"------- Epoch {epoch} -------")
            for x_batch, y_batch, c_batch in self.train_batches:
                x_batch, y_batch, c_batch = x_batch.to(self.device), y_batch.to(self.device), c_batch.to(self.device)
                tokens = self.model(x_batch)
                
                loss_calculater = LossCalculator(self.teacher_model)
                L_corrupt = None
                if len(tokens) == 3:
                    L_total, L_cls, L_distill, L_corrupt = loss_calculater((third_batch, y_batch, c_batch), tokens)
                else:
                    L_total, L_cls, L_distill = loss_calculater((third_batch, y_batch), tokens)

                # backprop
                optimizer.zero_grad()
                L_total.backward()
                optimizer.step()

                total_samples += y_batch.size(0)
                # losses
                loss_total += L_total.item() * y_batch.size(0)
                loss_cls += L_cls.item() * y_batch.size(0)
                loss_distill += L_distill.item() * y_batch.size(0)
                if L_corrupt : loss_corrupt += L_corrupt.item() * y_batch.size(0)

                # cosine similarity
                sim_cls_distill += self.model.sim_cls_distill_end
                sim_cls_corr += self.model.sim_cls_corr_end

            train_metrics = {
                "loss_total": loss_total/total_samples,
                "loss_cls": loss_cls/total_samples,
                "loss_distill": loss_distill/total_samples,
                "sim_cls_distill" : sim_cls_distill/total_samples,
                "sim_cls_corr" : sim_cls_corr/total_samples
            }
            if loss_corrupt : train_metrics["loss_corrupt"] = loss_corrupt/total_samples
            if self.print_metrics : print(f"Train-Loss: {train_metrics['loss_total']:.3f}")
            test_metrics = self.test_cdeit_model()
            self.all_train_metrics.append(train_metrics)
            self.all_test_metrics.append(test_metrics)
        
        # save trained model
        torch.save(self.model.state_dict(), self.save_path)
        print(f"\nModel saved to {self.save_path}")
